In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F

dtype = torch.FloatTensor

In [2]:
# Text-cnn 
embedding_size = 2 #n-gram
sequence_length = 3
num_classes = 2 # 0 or 1
filter_sizes = [2,2,2] # n-gram window
num_filters = 3

In [3]:
# sentences length is 3
sentences = ["i love you", "he loves me", "she likes baseball", "i hate you", "sorry for that", "this is awful"]
labels = [1, 1, 1, 0, 0, 0]  # 1 is good, 0 is not good.

In [14]:
word_list = " ".join(sentences).split()
word_list = list(set(word_list))
word_dict = {w:i for i, w in enumerate(word_list)}
num_dict = {i:w for i, w in enumerate(word_list)}
vocab_size = len(word_dict)

In [16]:
inputs = []
for s in sentences:
    inputs.append(np.asarray([word_dict[i] for i in s.split()]))

In [17]:
inputs

[array([11,  8, 14]),
 array([ 7,  6, 15]),
 array([ 0, 12,  9]),
 array([11,  1, 14]),
 array([ 5,  2, 10]),
 array([ 4,  3, 13])]

In [21]:
for i in inputs:
    print(num_dict[int(i[0])])

i
he
she
i
sorry
this


In [23]:
targets = []
for i in labels:
    targets.append(i)

In [25]:
input_batch = Variable(torch.LongTensor(inputs))
target_batch = Variable(torch.LongTensor(targets))

In [27]:
input_batch,target_batch

(tensor([[11,  8, 14],
         [ 7,  6, 15],
         [ 0, 12,  9],
         [11,  1, 14],
         [ 5,  2, 10],
         [ 4,  3, 13]]),
 tensor([1, 1, 1, 0, 0, 0]))

In [28]:
# model
class TextCNN(nn.Module):
    def __init__(self):
        super(TextCNN, self).__init__()

        self.num_filters_total = num_filters * len(filter_sizes)
        self.W = nn.Parameter(torch.empty(vocab_size, embedding_size).uniform_(-1, 1)).type(dtype)
        self.Weight = nn.Parameter(torch.empty(self.num_filters_total, num_classes).uniform_(-1, 1)).type(dtype)
        self.Bias = nn.Parameter(0.1 * torch.ones([num_classes])).type(dtype)

    def forward(self, X):
        embedded_chars = self.W[X] # [batch_size, sequence_length, sequence_length]
        embedded_chars = embedded_chars.unsqueeze(1) # add channel(=1) [batch, channel(=1), sequence_length, embedding_size]

        pooled_outputs = []
        for filter_size in filter_sizes:
            # conv : [input_channel(=1), output_channel(=3), (filter_height, filter_width), bias_option]
            conv = nn.Conv2d(1, num_filters, (filter_size, embedding_size), bias=True)(embedded_chars)
            h = F.relu(conv)
            # mp : ((filter_height, filter_width))
            mp = nn.MaxPool2d((sequence_length - filter_size + 1, 1))
            # pooled : [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3)]
            pooled = mp(h).permute(0, 3, 2, 1)
            pooled_outputs.append(pooled)

        h_pool = torch.cat(pooled_outputs, len(filter_sizes)) # [batch_size(=6), output_height(=1), output_width(=1), output_channel(=3) * 3]
        h_pool_flat = torch.reshape(h_pool, [-1, self.num_filters_total]) # [batch_size(=6), output_height * output_width * (output_channel * 3)]

        model = torch.mm(h_pool_flat, self.Weight) + self.Bias # [batch_size, num_classes]
        return model

In [30]:
model = TextCNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [31]:
# training
for epoch in range(5000):
    optimizer.zero_grad()
    out_put = model(input_batch)
    
    loss = criterion(out_put, target_batch)
    if (epoch+1) % 1000 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))

    loss.backward()
    optimizer.step()

Epoch: 1000 cost = 0.730619
Epoch: 2000 cost = 0.252034
Epoch: 3000 cost = 0.240010
Epoch: 4000 cost = 0.107126
Epoch: 5000 cost = 0.063472


In [43]:
# Test
# test_text = 'sorry hate you'
test_text = 'he hate she'
tests = [np.asarray([word_dict[n] for n in test_text.split()])]
test_batch = Variable(torch.LongTensor(tests))

In [44]:
# Predict
predict = model(test_batch).data.max(1, keepdim=True)[1]
if predict[0][0] == 0:
    print(test_text,"is Bad Mean...")
else:
    print(test_text,"is Good Mean!!")

he hate she is Good Mean!!
